### Download Sentinel-2 image collection

In [ ]:
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd
import ee

In [ ]:
#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)

file_path = './data/boundaries/isimangaliso_bbox.geojson'
with open(file_path) as f:
    bbox = geojson_to_ee(json.load(f))
    
Map.addLayer(ee_object=bbox, vis_params={'color':'red','fillColor': '00000000'}, name="Isimangaliso Wetland Park")
Map

In [ ]:
collection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(bbox).filterDate('2019-01-01','2019-01-30') \
    .filter(ee.Filter.inList('MGRS_TILE', ['36JVP','36JVQ'])) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 50)) \
    .select(['B4', 'B3', 'B2'])

print(f'Found:{collection.size().getInfo()} matches')

In [ ]:
collection_list = collection.toList(collection.size())

In [ ]:
latest_meta = collection_list.get(1)
geom = ee.Image(latest_meta).geometry()
tile = latest_meta.getInfo()['properties']['PRODUCT_ID'].split('_')[5]
Map.addLayer(geom , {'color': 'FF0000'}, tile)

In [ ]:
image = collection.mosaic().clip(bbox)
vizParams = {
    'bands': ['B4', 'B3', 'B2'],
    'gain': [0.1, 0.1, 0.1],
    'scale':10
};

Map.addLayer(image, vizParams, 'RGB mosaic')


In [ ]:
geemap.ee_export_image(image, filename='./data/test.tif', scale=10, region=bbox.geometry(), file_per_band=False)

In [ ]:
os.remove('./data/test.zip')

In [ ]:
def get_meta_s2(meta):
    product_id = meta['properties']['PRODUCT_ID']
    date = pd.to_datetime(product_id.split('_')[2].split('T')[0]).date()
    tile = product_id.split('_')[5]
    cloud_cover = round(meta['properties']['CLOUD_COVERAGE_ASSESSMENT'],2)
    data = {'id':product_id,'date':date,'tile':tile,'cloudcover':cloud_cover}
    df = pd.DataFrame(data,index=[0])
    return df

In [ ]:
products = list(map(get_meta_s2,collection_list.getInfo()))
products_df = pd.concat(products).reset_index(drop=True)
products_df